**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
suppressMessages(suppressWarnings(library("DESeq2")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


## Import count matrix and metadata to setup DESeq2

In [2]:
PREFIX   = "Tewhey_K562_TileMPRA"
FOLDER   = "coverage_astarrseq_peak_macs_input"

fdiry = file.path(FD_RES, "results", PREFIX, FOLDER, "summary")
print(dir(fdiry))

 [1] "matrix.raw.count.OL13_20220512.tsv"       
 [2] "matrix.raw.count.OL43_20221003.tsv"       
 [3] "matrix.raw.count.OL45_20220927.tsv"       
 [4] "matrix.raw.cpm.OL13_20220512.tsv"         
 [5] "matrix.raw.cpm.OL43_20221003.tsv"         
 [6] "matrix.raw.cpm.OL45_20220927.tsv"         
 [7] "metadata.raw.OL13_20220512.tsv"           
 [8] "metadata.raw.OL43_20221003.tsv"           
 [9] "metadata.raw.OL45_20220927.tsv"           
[10] "result.Log2FC.raw.cpm.Merge.tsv"          
[11] "result.Log2FC.raw.cpm.OL13_20220512.tsv"  
[12] "result.Log2FC.raw.cpm.OL43_20221003.tsv"  
[13] "result.Log2FC.raw.cpm.OL45_20220927.tsv"  
[14] "result.Log2FC.raw.deseq.Merge.tsv"        
[15] "result.Log2FC.raw.deseq.OL13_20220512.tsv"
[16] "result.Log2FC.raw.deseq.OL43_20221003.tsv"
[17] "result.Log2FC.raw.deseq.OL45_20220927.tsv"


In [3]:
PREFIX   = "Tewhey_K562_TileMPRA"
FOLDER   = "coverage_astarrseq_peak_macs_input"
DATASETS = c("OL13", "OL43", "OL45")

fdiry = file.path(FD_RES, "results", PREFIX, FOLDER, "summary")
for (DATASET in DATASETS){
    cat(DATASET, "\n")
    
    fname = paste("matrix.raw.count", DATASET, "tsv", sep = "*")
    fglob = file.path(fdiry, fname)
    fpath = Sys.glob(fglob)
    fname = basename(fpath)
    print(fname)
    
    fname = paste("metadata.raw", DATASET, "tsv", sep = "*")
    fglob = file.path(fdiry, fname)
    fpath = Sys.glob(fglob)
    fname = basename(fpath)
    print(fname)
}

OL13 
[1] "matrix.raw.count.OL13_20220512.tsv"
[1] "metadata.raw.OL13_20220512.tsv"
OL43 
[1] "matrix.raw.count.OL43_20221003.tsv"
[1] "metadata.raw.OL43_20221003.tsv"
OL45 
[1] "matrix.raw.count.OL45_20220927.tsv"
[1] "metadata.raw.OL45_20220927.tsv"


In [4]:
PREFIX   = "Tewhey_K562_TileMPRA"
FOLDER   = "coverage_astarrseq_peak_macs_input"
DATASETS = c(
    "OL13_20220512", 
    "OL43_20221003", 
    "OL45_20220927")

lst = lapply(DATASETS, function(DATASET){
    
    ### show progress
    cat("\n=============================\n")
    cat(DATASET, "\n")
    
    ### Import count matrix
    fname = paste("matrix.raw.count", DATASET, "tsv", sep = "*")
    fglob = file.path(fdiry, fname)
    fpath = Sys.glob(fglob)
    dat   = read_tsv(fpath, show_col_types = FALSE)
    
    cat("\n+++++++++++++++++++++++++++++\n")
    cat("Count matrix", "\n")
    dat_count = dat
    print(head(dat))
    
    ### Import metadata
    fname = paste("metadata.raw", DATASET, "tsv", sep = "*")
    fglob = file.path(fdiry, fname)
    fpath = Sys.glob(fglob)
    dat   = read_tsv(fpath, show_col_types = FALSE)
    
    cat("\n+++++++++++++++++++++++++++++\n")
    cat("Metadata", "\n")
    dat_meta = dat
    print(dat)
    
    ### Arrange count matrix and metadata
    dat_col = dat_meta  %>% 
        dplyr::select(Sample, Group) %>% 
        dplyr::rename(condition = Group) %>%
        column_to_rownames(var = "Sample")

    dat_cnt = dat_count %>% 
        column_to_rownames(var = "Peak")

    dat_cnt[is.na(dat_cnt)] = 0
    
    ### checking
    cat("\n+++++++++++++++++++++++++++++\n")
    cat("Check 01:", all(rownames(dat_col) %in% colnames(dat_cnt)), "\n")
    cat("Check 02:", all(rownames(dat_col) ==   colnames(dat_cnt)), "\n")
    
    ### create DESeq2 object
    dds = DESeqDataSetFromMatrix(
        countData = dat_cnt, 
        colData   = dat_col, 
        design    = ~condition)
    return(dds)
})

names(lst) = DATASETS
lst_dds_setup = lst


OL13_20220512 

+++++++++++++++++++++++++++++
Count matrix 
# A tibble: 6 × 9
  Peak       Input.rep1 Input.rep2 Input.rep3 Input.rep4 Output.rep1 Output.rep2
  <chr>           <dbl>      <dbl>      <dbl>      <dbl>       <dbl>       <dbl>
1 chr11:617…     123964      69072     113487      71526      461111      461000
2 chr11:618…      93429      53063      83490      53637      424590      398992
3 chr11:618…      73252      39188      63205      40131       29775       28918
4 chr11:618…     200996     114253     180968     113933     1392238     1406638
5 chr11:618…      31914      17567      28311      17860        8275        9393
6 chr11:618…      32681      18795      29971      19749       29360       29820
# ℹ 2 more variables: Output.rep3 <dbl>, Output.rep4 <dbl>

+++++++++++++++++++++++++++++
Metadata 
# A tibble: 8 × 3
  Sample      Group  FPath                                                      
  <chr>       <chr>  <chr>                                                

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”



OL43_20221003 

+++++++++++++++++++++++++++++
Count matrix 
# A tibble: 6 × 12
  Peak         Input.rep1 Input.rep2 Input.rep3 Input.rep4 Input.rep5 Input.rep6
  <chr>             <dbl>      <dbl>      <dbl>      <dbl>      <dbl>      <dbl>
1 chr8:126778…      10372      11304      10609      11042       5318       5475
2 chr8:126782…       3076       3477       3122       3211       1563       1614
3 chr8:126804…      16184      18791      17453      17754       8529       8533
4 chr8:126817…      10013      11400      10553      10900       5157       5076
5 chr8:126820…      16573      19018      17231      17757       8703       8698
6 chr8:126824…      27649      32054      29605      29659      14783      14827
# ℹ 5 more variables: Output.rep1 <dbl>, Output.rep2 <dbl>, Output.rep3 <dbl>,
#   Output.rep4 <dbl>, Output.rep5 <dbl>

+++++++++++++++++++++++++++++
Metadata 
# A tibble: 11 × 3
   Sample      Group  FPath                                                     
   <chr>   

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”



OL45_20220927 

+++++++++++++++++++++++++++++
Count matrix 
# A tibble: 6 × 9
  Peak       Input.rep1 Input.rep2 Input.rep3 Input.rep4 Output.rep1 Output.rep2
  <chr>           <dbl>      <dbl>      <dbl>      <dbl>       <dbl>       <dbl>
1 chr11:328…       5796       5349       5093       5236        4190        3932
2 chr11:328…       4925       4182       4315       4301        5557        3395
3 chr11:328…       8452       7263       7369       7154       12228       10838
4 chr11:328…      13661      11814      11553      11485       31756       29398
5 chr11:329…       8501       7376       7544       7221        6066        4803
6 chr11:329…      10373       8676       9442       9147        7865        6824
# ℹ 2 more variables: Output.rep3 <dbl>, Output.rep4 <dbl>

+++++++++++++++++++++++++++++
Metadata 
# A tibble: 8 × 3
  Sample      Group  FPath                                                      
  <chr>       <chr>  <chr>                                                

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


## Preprocess

**Pre-filtering**

In [5]:
lst = lst_dds_setup

lst = lapply(lst, function(dds){
    ### remove the peaks which have < 10 reads
    cat("Before filter:", nrow(dds), "\n")
    dds = dds[rowSums(counts(dds)) >= 10,]
    cat("After  filter:", nrow(dds), "\n")

    ### set control condition as reference
    dds$condition <- relevel(dds$condition, ref = "Input")
    return(dds)
})

lst_dds_filtered = lst

Before filter: 22 
After  filter: 22 
Before filter: 394 
After  filter: 394 
Before filter: 1306 
After  filter: 1306 


In [6]:
names(lst)

[1] "OL13_20220512" "OL43_20221003" "OL45_20220927"

## Run DESeq2

In [7]:
lst = lst_dds_filtered

lst = lapply(lst, function(dds){
    ### apply DESeq2 data processing
    dds = DESeq(dds, fitType='local')
    return(dds)
})

lst_dds_run = lst

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



## Get results

In [8]:
lst = lst_dds_run

lst = lapply(lst, function(dds){
    res = results(dds)
    res = as.data.frame(res) %>% rownames_to_column(var = "Peak")
    return(res)
})

lst_res = lst
print(names(lst))

[1] "OL13_20220512" "OL43_20221003" "OL45_20220927"


## Save results

In [9]:
lst = lst_res

for (idx in names(lst)){
    cat("\n+++++++++++++++++++++++++++++\n")
    cat(idx, "\n")
    
    fdiry = file.path(FD_RES, "results", PREFIX, FOLDER, "summary")
    fname = paste("result.Log2FC.raw.deseq", idx, "tsv", sep = ".")
    fpath = file.path(fdiry, fname)
    print(fpath)
    
    res = lst[[idx]]
    write_tsv(res, fpath)
}


+++++++++++++++++++++++++++++
OL13_20220512 
[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/coverage_astarrseq_peak_macs_input/summary/result.Log2FC.raw.deseq.OL13_20220512.tsv"

+++++++++++++++++++++++++++++
OL43_20221003 
[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/coverage_astarrseq_peak_macs_input/summary/result.Log2FC.raw.deseq.OL43_20221003.tsv"

+++++++++++++++++++++++++++++
OL45_20220927 
[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/coverage_astarrseq_peak_macs_input/summary/result.Log2FC.raw.deseq.OL45_20220927.tsv"
